Initial setup

In [3]:
%load_ext autoreload
%autoreload 2

#modules for setting the path
import os
import sys
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

#modules for opening the data
import psycopg2
import pandas as pd
from src.data import local

#Retrieving postgres info
PORT = local.port
USER = local.user
HOST = local.host
PASSWORD = local.password

DBNAME = "opportunity_youth"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
conn = psycopg2.connect(dbname=DBNAME, user=USER, password=PASSWORD, port=PORT, host=HOST)

First I want to find out what is in the puma_names_2010 table.

In [9]:
query = """
SELECT *
FROM puma_names_2010
"""

puma_names_table = pd.read_sql(query, conn)

puma_names_table.head()

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name
0,1,Alabama ...,1,100,100100,G01000100,"Lauderdale, Colbert, Franklin & Marion (Northe..."
1,1,Alabama ...,1,800,100800,G01000800,St. Clair & Blount Counties ...
2,1,Alabama ...,1,1400,101400,G01001400,"Walker, Marion (South & West), Fayette & Lamar..."
3,1,Alabama ...,1,1500,101500,G01001500,Tuscaloosa (Outer) & Pickens Counties--Northpo...
4,1,Alabama ...,1,1600,101600,G01001600,Tuscaloosa & Northport (Southeast) Cities ...


Let's see if we can find south king county

In [10]:
query = """
SELECT *
FROM puma_names_2010
WHERE puma_name = 'South King County'
"""

skc_search = pd.read_sql(query, conn)

skc_search.head()

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name


In [11]:
query = """
SELECT *
FROM puma_names_2010
WHERE puma_name LIKE '%King%'
"""

skc_search = pd.read_sql(query, conn)

skc_search.head()

,state_fips,state_name,cpuma0010,puma,geoid,gisjoin,puma_name
0,6,California ...,69,3100,603100,G06003100,Kings County--Hanford City ...
1,25,Massachusetts ...,457,4903,2504903,G25004903,"Plymouth County (East)--Plymouth, Marshfield, ..."
2,36,New York ...,702,3701,3603701,G36003701,"NYC-Bronx Community District 8--Riverdale, Fie..."
3,42,Pennsylvania ...,862,801,4200801,G42000801,Luzerne County (East)--Kingston Borough ...
4,42,Pennsylvania ...,889,3103,4203103,G42003103,Montgomery County (Southwest)--King of Prussia...


It doesn't look like South King County is in this table, so let's check wa_geo_xwalk

In [15]:
query = """
SELECT *
FROM wa_geo_xwalk
"""

skc_search = pd.read_sql(query, conn)

skc_search.columns

Index(['tabblk2010', 'st', 'stusps', 'stname', 'cty', 'ctyname', 'trct',
       'trctname', 'bgrp', 'bgrpname', 'cbsa', 'cbsaname', 'zcta', 'zctaname',
       'stplc', 'stplcname', 'ctycsub', 'ctycsubname', 'stcd116',
       'stcd116name', 'stsldl', 'stsldlname', 'stsldu', 'stslduname',
       'stschool', 'stschoolname', 'stsecon', 'dtseconname', 'trib',
       'tribname', 'tsub', 'tsubname', 'stanrc', 'stanrcname', 'necta',
       'nectname', 'mil', 'milname', 'stwib', 'stwibname', 'blklatdd',
       'blklondd', 'createdate'],
      dtype='object')

In [16]:
skc_search.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 195574 entries, 0 to 195573
Data columns (total 43 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   tabblk2010    195574 non-null  object 
 1   st            195574 non-null  object 
 2   stusps        195574 non-null  object 
 3   stname        195574 non-null  object 
 4   cty           195574 non-null  object 
 5   ctyname       195574 non-null  object 
 6   trct          195574 non-null  object 
 7   trctname      195574 non-null  object 
 8   bgrp          195574 non-null  object 
 9   bgrpname      195574 non-null  object 
 10  cbsa          195574 non-null  object 
 11  cbsaname      175100 non-null  object 
 12  zcta          195574 non-null  object 
 13  zctaname      192838 non-null  object 
 14  stplc         195574 non-null  object 
 15  stplcname     113097 non-null  object 
 16  ctycsub       195574 non-null  object 
 17  ctycsubname   195574 non-null  object 
 18  stcd